In [1]:
import json
import numpy as np
import os
import pandas as pd

cwd = os.getcwd()
pkg_dir = os.path.dirname(cwd)
data_dir = os.path.join(pkg_dir, 'tests/data')

from reV.handlers.transmission import TransmissionFeatures
from reV.supply_curve.supply_curve import SupplyCurve

In [2]:
path = os.path.join(data_dir, 'sc_tables', '2_ri_agg_points_cache_064_sj_infsink.csv')
sc_table = pd.read_csv(path)

trans_feats = TransmissionFeatures(sc_table)

path = os.path.join(data_dir, 'sc_out', 'baseline_agg_summary.csv')
sc_points = pd.read_csv(path).rename(columns={'sc_gid': 'sc_point_gid'})
sc_points['sc_gid'] = sc_points.index

sc = SupplyCurve(sc_points, sc_table)
sc._serial_sort()
serial_final = sc._sc_points.copy()

In [31]:
sc = SupplyCurve(sc_points, sc_table)
sc_mapping = sc._sc_table.sort_values('lcot')
i = 0

In [33]:
avail = sc._trans_features._mask
avail = avail.index[avail['available'].values]
empty = sc._mask
empty = empty.index[empty['empty'].values]
print('emtpy = {}'.format(len(empty)))

pos = sc_mapping['sc_gid'].isin(empty)
to_connect = sc_mapping.loc[pos].groupby('sc_gid').first().reset_index()

pos = to_connect['trans_line_gid'].isin(avail)
to_connect = to_connect.loc[pos].groupby('trans_line_gid').first().reset_index()

for _, row in to_connect.iterrows():
    sc_gid = row['sc_gid']
    if sc._mask.loc[sc_gid, 'empty']:
        trans_gid = row['trans_line_gid']
        connect = sc._trans_features.connect(trans_gid,
                                               row['capacity'])
        if connect:
            sc._mask.loc[sc_gid, 'empty'] = False
            sc._sc_points.at[sc_gid, 'trans_gid'] = trans_gid
            sc._sc_points.at[sc_gid, 'trans_type'] = row['category']
            sc._sc_points.at[sc_gid, 'lcot'] = row['lcot']
    print(sc_gid, connect)

print('full = {}'.format(np.sum(~sc._mask['empty'])))
    

emtpy = 141
25 False
100 False
11 False
2 False
3 False
5 False
17 False
34 False
42 False
36 False
66 False
55 False
77 False
79 False
38 False
29 False
81 False
49 False
60 False
105 False
95 False
83 False
72 False
85 False
73 False
86 False
46 False
56 False
47 False
10 False
27 False
87 False
26 False
6 False
51 False
116 False
118 False
69 False
58 False
full = 2


In [23]:
sc = SupplyCurve(sc_points, sc_table)
sc_mapping = sc._sc_table.sort_values('lcot')
i = 0

while sc._mask['empty'].any():
    avail = sc._trans_features._mask
    avail = avail.index[avail['available'].values]
    empty = sc._mask
    empty = empty.index[empty['empty'].values]

    pos = sc_mapping['sc_gid'].isin(empty)
    to_connect = sc_mapping.loc[pos].groupby('sc_gid').first().reset_index()

    pos = to_connect['trans_line_gid'].isin(avail)
    to_connect = to_connect.loc[pos].groupby('trans_line_gid').first().reset_index()
    sc._serial_sort(sc_table=to_connect)
    print('step {}: {} of {} sites full'
          .format(i, np.sum(~sc._mask['empty']), len(sc._mask)))
    i += 1

step 0: 2 of 143 sites full
step 1: 2 of 143 sites full
step 2: 2 of 143 sites full
step 3: 2 of 143 sites full
step 4: 2 of 143 sites full
step 5: 2 of 143 sites full
step 6: 2 of 143 sites full
step 7: 2 of 143 sites full
step 8: 2 of 143 sites full
step 9: 2 of 143 sites full
step 10: 2 of 143 sites full
step 11: 2 of 143 sites full
step 12: 2 of 143 sites full
step 13: 2 of 143 sites full
step 14: 2 of 143 sites full
step 15: 2 of 143 sites full
step 16: 2 of 143 sites full
step 17: 2 of 143 sites full
step 18: 2 of 143 sites full
step 19: 2 of 143 sites full
step 20: 2 of 143 sites full
step 21: 2 of 143 sites full
step 22: 2 of 143 sites full
step 23: 2 of 143 sites full
step 24: 2 of 143 sites full
step 25: 2 of 143 sites full
step 26: 2 of 143 sites full
step 27: 2 of 143 sites full
step 28: 2 of 143 sites full
step 29: 2 of 143 sites full
step 30: 2 of 143 sites full
step 31: 2 of 143 sites full
step 32: 2 of 143 sites full
step 33: 2 of 143 sites full
step 34: 2 of 143 sites 

KeyboardInterrupt: 